In [1]:
!pip install evodiff
!pip install torch_geometric
!pip install torch_scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from evodiff.pretrained import OA_DM_640M
checkpoint = OA_DM_640M()
model, collater, tokenizer, scheme = checkpoint
model.to("cuda")

Downloading: "https://zenodo.org/record/8045076/files/oaar-640M.tar?download=1" to /root/.cache/torch/hub/checkpoints/oaar-640M.tar
100%|██████████| 7.19G/7.19G [06:39<00:00, 19.3MB/s]


ByteNetLMTime(
  (embedder): ByteNetTime(
    (time_encoding): PositionalEncoding1D()
    (embedder): Embedding(31, 8, padding_idx=28)
    (up_embedder): PositionFeedForward(
      (conv): Conv1d(8, 1280, kernel_size=(1,), stride=(1,))
    )
    (layers): ModuleList(
      (0): ByteNetBlock(
        (conv): MaskedConv1d(1280, 1280, kernel_size=(5,), stride=(1,), padding=(2,))
        (sequence1): Sequential(
          (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (1): GELU(approximate='none')
          (2): PositionFeedForward(
            (conv): Conv1d(1280, 1280, kernel_size=(1,), stride=(1,))
          )
          (3): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (4): GELU(approximate='none')
        )
        (sequence2): Sequential(
          (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (1): GELU(approximate='none')
          (2): PositionFeedForward(
            (conv): Conv1d(1280, 1280, kernel_size=(1,), stri

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
from time import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/IL10_scaffolding/evodiff_il10/generation_metadata_evodiff_tev.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created generation metadata dataframe")



Mounted at /content/drive
Existing generation metadata read in.


In [4]:
import time
"""
The default generate_scaffold method in evodiff conditional_generation.py places the motif randomly in the scaffold
which is not aligned with out task. I've rewritten the code in order to keep the location of the motif static.
It also can only take pdb ids rather than raw files, but it looks like they just extract the sequence anyway so
we'll just specify the seq.
"""

from evodiff.generate import generate_oaardm
from evodiff.conditional_generation import generate_scaffold

import evodiff
from evodiff.pretrained import OA_DM_640M, OA_DM_38M, CARP_640M, LR_AR_38M, LR_AR_640M
import numpy as np
import argparse
import urllib.request
import torch
import os
import esm.inverse_folding
from evodiff.utils import Tokenizer, run_omegafold, clean_pdb, run_tmscore #, wrap_dr_bert, read_dr_bert_output
import pathlib
from sequence_models.utils import parse_fasta
from tqdm import tqdm
import pandas as pd
import random
from evodiff.plot import aa_reconstruction_parity_plot, idr_parity_plot

for k in range(0,5):
  batch_size = 10
  torch.cuda.empty_cache()
  meta_data = {}
  meta_data["batch_id"] = str(uuid.uuid4())
  meta_data["batch_size"] = str(batch_size)
  meta_data["Timestamp"] = str(datetime.now())
  meta_data["model"] = "evodiff_OA_DM_640M"
  meta_data["task"] = "sequence_generation"
  #meta_data["conditions"] = "IL10 (monomer) scaffolding [motif_seq = QSENSCTHFPGYLPNMLRDLRDAFSR########################################LEEVMPQAENQDPDIKAHVISLGENLNTLRLRLRR, seq_len = 184, start = 24]"
  meta_data["conditions"] = "tev (monomer) scaffolding [motif_seq = GHTTSL#############HLFRR########################################################################################FWKHWIQTKDGQC###############HSASNFTNTNNY################################WGGHKVFMVK, seq_len = 237, start = 27]"
  meta_data["gpu"] = "T4 GPU"
  meta_data['output_file_name'] = None

  num_seqs = batch_size
  #motif_seq = "QSENSCTHFPGYLPNMLRDLRDAFSR########################################LEEVMPQAENQDPDIKAHVISLGENLNTLRLRLRR"
  motif_seq = "GHTTSL#############HLFRR########################################################################################FWKHWIQTKDGQC###############HSASNFTNTNNY################################WGGHKVFMVK"


  device="cuda"
  #seq_len = 184
  seq_len = 237
  #start = 24
  start = 27

  start_time = time.time()

  mask = tokenizer.mask_id
  motif_tokenized = tokenizer.tokenize((motif_seq,))
  sample = torch.zeros((num_seqs, seq_len)) + mask # start from all mask
  sample[:, start:start+len(motif_seq)] = torch.tensor(motif_tokenized)
  nonmask_locations = (sample[0] != mask).nonzero().flatten()
  value, loc = (sample == mask).long().nonzero(as_tuple=True) # locations that need to be unmasked
  loc = np.array(loc)
  np.random.shuffle(loc)
  sample = sample.long().to(device)
  with torch.no_grad():
    for i in loc:
      timestep = torch.tensor([0] * num_seqs)  # placeholder but not called in model
      timestep = timestep.to(device)
      prediction = model(sample, timestep)
      p = prediction[:, i, :len(tokenizer.all_aas) - 6]  # only canonical
      p = torch.nn.functional.softmax(p, dim=1)  # softmax over categorical probs
      p_sample = torch.multinomial(p, num_samples=1)
      sample[:, i] = p_sample.squeeze()
  untokenized = [tokenizer.untokenize(s) for s in sample]
  end_time = time.time()
  total_job_time = end_time - start_time
  meta_data["wall_time_batch"] = str(total_job_time) + " Seconds"
  meta_data['wall_time_task'] = str(total_job_time/batch_size) + " Seconds (inferred)"

  print("Generated sequence:", untokenized)

  for seq in untokenized:
    meta_data['entity_id'] = str(uuid.uuid4())
    meta_data['generated_sequence'] = seq
    metadata_entry = pd.Series(meta_data)
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(metadata_entry).T], ignore_index=True)
  all_metadata_df.to_csv(meta_data_filepath, index=False)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Generated sequence: ['VVETPVTTPNKAGFAHPFLMAVRDALTGHTTSLAKGVPKDPVEFQWHLFRRLNSEPALLAGITKLDEPGKRLYEVWHGQNVHIGDRSHLILIDIDKPSLQPILEAYSHQAPVHNHPMAPYLAPIRRRAHMLDSAKDDWAFWKHWIQTKDGQCHICLRSHYLRESLPKHSASNFTNTNNYAIVAGSRWATGKLAEVICAGCLNELREGFIKDWGGHKVFMVKNSLAVEGRLRRRQRRR', 'MRRKTELSVSSTSPTLTDYRNTELNKYGHTTSLLNNYKNHLKRTVPHLFRRPDELDPMQWAIRLNSTLTHTAALKKHFSAYVRCNCLSHLRQLNFHKSLEEDMNTEELSTNKKKDQPWLKGMKSKTNQREITPACRTYAFWKHWIQTKDGQCATKKCMKILRQCTGNHSASNFTNTNNYIMACEESITYNSNPYTKVSKCTKRAFMSFHITWGGHKVFMVKRGWIFCNVTGFLRWLS', 'MDFSQSPVSSTNTRERFLLVKTEIDMIGHTTSLRCYIQSANEKDSGHLFRRTKPSLADCDLILIRIPSYIWVKTVRVPLARYYMGTTSRQHKIMSTKLRSYVCCDAKSETNALTEMHRTPGPSRVGLHRIIAKLDKVYSFWKHWIQTKDGQCLFNDGWFRGQSGQYIHSASNFTNTNNYVSEFHHFIGCLMLSSKFNVARSYIREEISVLLWGGHKVFMVKRSEHRRGVNWVTTHPT', 'MDKGYKQLDKTIAKIPYVYTNYNFIILGHTTSLYATFNKIYFKFVKHLFRRKIPASKIRLVKLYPINNNSDEEMYIREVIERRKTLCESDIAFVYYKTNVNKLRFSRIDSRPFFFNLLPTLDYVDKHTRDRIRASVLPNFWKHWIQTKDGQCYENLSKVENAKGLNFHSASNFTNTNNYYNMAVLPRNRDDEQLIQFIFDKSMQTTSPFMNWGGHKVFMVKMKDLPSLKERRKLSIL', 'MFAYSSCIIRGKWN

In [ ]:
# prompt: disconnect and delete the colab runtime

!kill -9 -1


In [ ]:
#Depreciated VVVVVVVVV

In [ ]:
!export PYTHONPATH=""
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
%%bash
source activate evodiff
pip install evodiff
conda install pytorch torchvision torchaudio -c pytorch
conda install pyg -c pyg
conda install -c conda-forge torch-scatter